In [ ]:
## Change these variables to your desired values
img_height = 256
img_width = 256
img_channels = 1

path_videos = 'assets/video_data_3/'
path_out_images = 'assets/labeled_photos_3(cropped)/'

batch_size = 16

label_names=['Human',
             'Interaction frontal',
             'Interaction lateral', 
             'Interaction vertical',
             'Crowded', 
             'Drink',
             'Curiosity', 
             'Queue',
             'Low visibility', 
             'Nothing']
layer_names = ['conv2d_1', 
               'conv2d_2', 
               'conv2d_3', 
               'conv2d_4', 
               'conv2d_5', 
               'conv2d_6', 
               'conv2d_7']

In [ ]:
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

from keras import backend as K
from keras.applications.vgg16 import VGG16
import cv2

In [ ]:
def preprocess_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) # convert to greyscale
    frame = cv2.resize (frame, (img_width, img_height), interpolation=cv2.INTER_CUBIC) # rezize
    return frame

def frame_from_pointer (path, video, frame, verbose=False):
    if verbose: print ('In '+path+'/'+str(video)+'.mp4' + ', taking frame ' + str(frame))
    cap = cv2.VideoCapture(path+'/'+str(video)+'.mp4')
    if (cap.isOpened()== False):
        print("Error opening video file") 
        return -1
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1)
    ret, frame = cap.read() # Capture next frame
    if ret==True:
        cap.release()
        return frame
    else:
        print("Error opening frame") 
        return -1


In [ ]:
from keras.models import load_model
model = load_model('assets/models/STABLE_SR2_november29.h5')

# Grad-CAM
I'm using Keras-vis library

In [ ]:
import keras

from vis.utils import utils
import vis 
from vis.visualization import visualize_cam

In [ ]:
def plot_map(grads, _img, label="undefined", layer="last"):
    fig, axes = plt.subplots(1,2,figsize=(14,5))
    axes[0].imshow(_img)
    axes[1].imshow(_img)
    i = axes[1].imshow(grads,cmap="jet",alpha=0.8)
    fig.colorbar(i)
    plt.suptitle("Class = {} \nLayer = {}".format(
                      label,
                      layer))
def preprocess_heatmap(grad_top, target_size):
    # We resize the heatmap to have the same size as the original image
    grad_top = cv2.resize(grad_top, target_size)

    # Convert to Grayscale (following the keras-viz pattern)
    heatmap = 2.0*grad_top[:,:,0] + 2.0*grad_top[:,:,1] - 1.0*grad_top[:,:,2]
    heatmap += abs(min(heatmap.min(), 0))
    if heatmap.max(): heatmap /= heatmap.max()
    heatmap *= 255
    heatmap = heatmap.astype('uint8')
    
    return heatmap

In [ ]:
layer_idx = utils.find_layer_idx(model, 'dense_3')
# Swap softmax with linear
model.layers[layer_idx].activation = keras.activations.linear
model = utils.apply_modifications(model)

In [ ]:
frame_original = frame_from_pointer('assets/video_data_3',1569153540, 6229, verbose=True)
plt.imshow(frame_original)

frame = preprocess_frame (frame_original)
frame = frame.reshape(frame.shape[0], frame.shape[1], img_channels)
frame = np.expand_dims(frame, axis=0)

y_hat = model.predict(frame)
print(y_hat)

In [ ]:
class_idx = 2

layer = layer_names[-3]
grad_top3  = visualize_cam(model, layer_idx, class_idx, 
                           seed_input = frame,
                           penultimate_layer_idx = utils.find_layer_idx(model, layer),
                           backprop_modifier     = None,
                           grad_modifier         = None)
plot_map(grad_top3, frame.reshape(frame.shape[1], frame.shape[2]), label_names[class_idx], layer)

layer = layer_names[-2]
grad_top2  = visualize_cam(model, layer_idx, class_idx, 
                           seed_input = frame,
                           penultimate_layer_idx = utils.find_layer_idx(model, layer),
                           backprop_modifier     = None,
                           grad_modifier         = None)
plot_map(grad_top2, frame.reshape(frame.shape[1], frame.shape[2]), label_names[class_idx], layer)

layer = layer_names[-1]
grad_top1  = visualize_cam(model, layer_idx, class_idx, 
                           seed_input = frame,
                           penultimate_layer_idx = utils.find_layer_idx(model, layer),
                           backprop_modifier     = None,
                           grad_modifier         = None)
plot_map(grad_top1, frame.reshape(frame.shape[1], frame.shape[2]), label_names[class_idx], layer)

# Vizualizing every filter
Reference: https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.4-visualizing-what-convnets-learn.ipynb

In [ ]:
from keras import models

# Extracts the outputs of the top 8 layers:
layer_outputs = [layer.output for layer in model.layers[:14]]
# Creates a model that will return these outputs, given the model input:
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

# This will return a list of 5 Numpy arrays:
# one array per layer activation
activations = activation_model.predict(frame)
len(activations)

In [ ]:
# These are the names of the layers, so can have them as part of our plot
layer_names = []
for layer in model.layers[:14]:
    layer_names.append(layer.name)

images_per_row = 16

# Now let's display our feature maps
for layer_name, layer_activation in zip(layer_names, activations):
    # This is the number of features in the feature map
    n_features = layer_activation.shape[-1]

    # The feature map has shape (1, size, size, n_features)
    size = layer_activation.shape[1]

    # We will tile the activation channels in this matrix
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # We'll tile each filter into this big horizontal grid
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            # Post-process the feature to make it visually palatable
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image

    # Display the grid
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

License: Creative Commons 4.0 Attribute